In [ ]:
import cv2
import numpy as np
import tkinter as tk
import dlib
import math
from tkinter import filedialog
class SpeedCalculatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Speed Detector")
        self.root.geometry("800x400")

        # Create GUI elements
        self.title_label = tk.Label(root, text="Speed Detector for Cars", font=("Helvetica", 30, "bold"))
        self.title_label.pack(pady=10)

        self.subtitle_label = tk.Label(root, text="-------", font=("Helvetica", 16))
        self.subtitle_label.pack(pady=10)

        self.selected_option = tk.StringVar(value="video")  # default option is video

        self.video_radio = tk.Radiobutton(root, text="Video", variable=self.selected_option, value="video")
        self.video_radio.pack(pady=5, anchor='center')

        self.camera_radio = tk.Radiobutton(root, text="Real-time Camera", variable=self.selected_option, value="camera")
        self.camera_radio.pack(pady=5, anchor='center')

        self.video_path = tk.StringVar()
        self.video_entry = tk.Entry(root, textvariable=self.video_path, width=40)
        self.video_entry.pack(pady=5)

        self.browse_button = tk.Button(root, text="Browse", command=self.browse_video)
        self.browse_button.pack(pady=10)

        self.find_speed_button = tk.Button(root, text="Find Speed", command=self.find_speed)
        self.find_speed_button.pack(pady=10)

        self.result_label = tk.Label(root, text="", font=("Helvetica", 16, "bold"), fg="green")
        self.result_label.pack(pady=10)
        self.rectangleColor = (104, 43, 159)
        # speed tracking variables
        self.carTracker = {}
        self.carLocation1 = {}
        self.carLocation2 = {}
        self.final = 0
        self.speed = [None] * 10000
        self.frameCounter = 0
        self.currentCarID = 0
        self.total_displacement = 0
        self.pixels_per_cm = 16.16  # we adjust this value based on video

    def browse_video(self):
        if self.selected_option.get() == "video":
            file_path = filedialog.askopenfilename()
            if file_path:
                self.video_path.set(file_path)

    def find_speed(self):
        video_path = self.video_path.get()

        # video parameters
        frame_rate = 30  # frames per second
        fps = 30  # frames per second for video processing

        # load video or real-time camera
        if self.selected_option.get() == "camera":
            cap = cv2.VideoCapture(0)  # Use webcam
        else:
            cap = cv2.VideoCapture(video_path)

            # checking if video opened successfully
            if not cap.isOpened():
                print("Error opening video file")
                return

   
        def process_frame():
            # reading next frame
            finalSpeed=0
            ret, frame = cap.read()
            if not ret:
                cap.release()
                self.display_result(finalSpeed)
                return

            frame = cv2.resize(frame, (1280, 720))
            resultImage = frame.copy()

            self.frameCounter = self.frameCounter + 1
            carIDtoDelete = []

            # convert the frame to HSV
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # creating a mask to filter out the black color of car
            lower_black = np.array([0, 0, 0])
            upper_black = np.array([180, 255, 30])
            mask = cv2.inRange(hsv, lower_black, upper_black)

            # finding contours in the mask
            contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for carID in self.carTracker.keys():
                trackingQuality = self.carTracker[carID].update(frame)

                if trackingQuality < 7:
                    carIDtoDelete.append(carID)

            for carID in carIDtoDelete:
                self.carTracker.pop(carID, None)
                self.carLocation1.pop(carID, None)
                self.carLocation2.pop(carID, None)

            if not (self.frameCounter % 10):
                # process the contours
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    x_bar = x + 0.5 * w
                    y_bar = y + 0.5 * h

                    matchCarID = None

                    for carID in self.carTracker.keys():
                        trackedPosition = self.carTracker[carID].get_position()

                        t_x = int(trackedPosition.left())
                        t_y = int(trackedPosition.top())
                        t_w = int(trackedPosition.width())
                        t_h = int(trackedPosition.height())

                        t_x_bar = t_x + 0.5 * t_w
                        t_y_bar = t_y + 0.5 * t_h

                        if (
                            (t_x <= x_bar <= (t_x + t_w))
                            and (t_y <= y_bar <= (t_y + t_h))
                            and (x <= t_x_bar <= (x + w))
                            and (y <= t_y_bar <= (y + h))
                        ):
                            matchCarID = carID

                    if matchCarID is None:
                        tracker = dlib.correlation_tracker()
                        tracker.start_track(frame, dlib.rectangle(x, y, x + w, y + h))

                        self.carTracker[self.currentCarID] = tracker
                        self.carLocation1[self.currentCarID] = [x, y, w, h]

                        self.currentCarID = self.currentCarID + 1

            for carID in self.carTracker.keys():
                trackedPosition = self.carTracker[carID].get_position()

                t_x = int(trackedPosition.left())
                t_y = int(trackedPosition.top())
                t_w = int(trackedPosition.width())
                t_h = int(trackedPosition.height())

                cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), self.rectangleColor, 2)

                self.carLocation2[carID] = [t_x, t_y, t_w, t_h]
     
            for i in self.carLocation1.keys():
                if self.frameCounter % 1 == 0:
                    [x1, y1, w1, h1] = self.carLocation1[i]
                    [x2, y2, w2, h2] = self.carLocation2[i]

                    self.carLocation1[i] = [x2, y2, w2, h2]

                    if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                        if (self.speed[i] == None or self.speed[i] == 0) and y1 >= 275 and y1 <= 285:
                            self.speed[i] = self.estimate_speed([x1, y1, w1, h1], [x2, y2, w2, h2])
                        if self.speed[i] is not None and y1 >= 180:
                            finalSpeed=self.speed[i]
                            self.final=finalSpeed
                            print("fin",finalSpeed)
                            result_text = f"Object Speed: {self.final} km/hr"
                            self.result_label.config(text=result_text, fg="white")
                            speed_text = f"{int(self.speed[i])} km/h"
                            cv2.putText(resultImage, speed_text, (int(x1 + w1/2), int(y1 - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 255, 255), 2)

            # display the image after processing all objects
            cv2.imshow('Speed Calculator', resultImage)
   


            if cv2.waitKey(1) == 1:
                cap.release()
                cv2.destroyAllWindows()
                self.display_result(finalSpeed)
            else:
                # schedule the next frame processing
                self.root.after(2, process_frame)

        # start processing frames
        process_frame()

    def estimate_speed(self, location1, location2):
        fps=18
        ppm=16.16
        d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
        d_meters = d_pixels / ppm
        speed = d_meters * fps * 3.6  # convert to km/h
        print("speed:", speed)
        return speed

    def display_result(self, finalSpeed):      
        result_text = f"Car Speed: {self.final} km/hr"
        self.result_label.config(text=result_text, fg="white")


if __name__ == "__main__":
    root = tk.Tk()
    app = SpeedCalculatorApp(root)
    root.mainloop()


2023-12-03 22:36:51.741 python[81244:11417823] +[CATransaction synchronize] called within transaction


speed: 28.35428181985661
fin 28.35428181985661
fin 28.35428181985661
fin 28.35428181985661
fin 28.35428181985661
fin 28.35428181985661
speed: 36.969460447560586
fin 36.969460447560586
fin 28.35428181985661
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
speed: 0.0
fin 0.0
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
fin 36.969460447560586
